In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

#variable

SHORTEN = {
    "Artery-Aorta":"Artery.A"     ,
    "Artery-Tibial": "Artery.T",
    "Adipose-Subcutaneous": "Adipose.S",    
    "Adipose-Visceral":"Adipose.V",
    "Brain-Caudate":"Caudate"   , 
    "Brain-Cerebellum":"Cerebellum",
    "Cells-Transformedfibroblasts": "Fibroblast",
    "Esophagus-Mucosa": "Mucosa",
    "Esophagus-Muscularis":"Muscularis",
    "Heart-LeftVentricle":"Heart",
    "Lung": "Lung",
    "Muscle-Skeletal": "Muscle",
    "Nerve-Tibial":"Nerve",
    "Skin-NotSunExposed": "SkinUnexposed",
    "Skin-SunExposed":"SkinLeg",
    "Thyroid":"Thyroid",
    "WholeBlood": "Blood"
}
COLORS = {
    "Artery-Aorta":"salmon",
    "Artery-Tibial": "red",
    "Adipose-Subcutaneous": "darkorange",    
    "Adipose-Visceral":"orange",
    "Brain-Caudate":"lemonchiffon"   , 
    "Brain-Cerebellum":"yellow",
    "Cells-Transformedfibroblasts": "skyblue",
    "Esophagus-Mucosa": "sienna",
    "Esophagus-Muscularis":"burlywood",
    "Heart-LeftVentricle":"darkviolet",
    "Lung": "greenyellow",
    "Muscle-Skeletal": "mediumslateblue",
    "Nerve-Tibial":"gold",
    "Skin-NotSunExposed":"blue",
    "Skin-SunExposed":"cornflowerblue",
    "Thyroid":"green",
    "WholeBlood": "m",
    "permuted": "gray"
}
TISSUES = sorted([item for item in list(SHORTEN.keys()) if item != "permuted"])
SUMMARY = '/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/LR_SummaryTest_Table.tsv'
CAUSESCORE='/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/Merged_Best_causality.Table'
ANNOT = '/storage/resources/dbase/human/hg19/gencode_gene_annotations_hg19.csv'
TSS = '/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/Merged_Causality_TSS_TSS.tab'
OUTPATH = "pdfs"
PREFIX = "localization"
ANNOTDIR = "/storage/mgymrek/gtex/annotations/"
                #exon
#location       #intron
                #CDS (coding region)
                #UTR
#sep='"\t"'
#command = "cat %s |awk -F'\t' '{print $1%s$2%s$75%s$4}' > all_analyzed_strs.tab"%(SUMMARY, sep,sep,sep)
#value = os.system(command)
allstrs = pd.read_csv(SUMMARY, sep='\t')
allstrs['locus']=allstrs.apply(lambda x: x["chrom"]+":"+str(x["str.start"]), 1)

In [16]:
#strs at location regions
##                               Location
locations = ['introns','coding','5utr','3utr','exon']
table={}
feature_count=[]
hom_feature_count=[]
specs=['Num.str','Num.estr','Num.hom','Num.e.hom']
for location in locations:

    command="bedtools intersect -a all_analyzed_strs.tab -b %s%s.bed -wa > intersect.tab"%(ANNOTDIR, location)
    value=os.system(command)

    feature_strs=pd.read_csv('intersect.tab', header=None, sep='\t', names=['chrom','str.start','str.end','gene'])
    feature_strs = feature_strs.merge(allstrs, on=['chrom','str.start','str.end','gene'], how='inner')
    Hom = feature_strs.loc[feature_strs['period']==1]
    estrs_feat=feature_strs.loc[feature_strs['E.tissues']>0]
    ehom = estrs_feat.loc[estrs_feat['period']==1]
    table[location] = [len(set(feature_strs["locus"])), len(set(estrs_feat['locus'])),len(set(Hom['locus'])),len(set(ehom['locus']))]
    print("\nTotal STR loci in %s region: %s"%(location, str(len(set(feature_strs['locus'])))))
    print("Total eSTRs found in %s region: %s"%(location, str(len(set(estrs_feat['locus'])))))
    feature_count.append(feature_strs)
    hom_feature_count.append(ehom)
estrs=allstrs.loc[allstrs['E.tissues']>0]
table['All.loci']=[len(set(allstrs['locus'])),len(set(estrs['locus'])),len(set(allstrs.loc[allstrs['period']==1]['locus'])),len(set(estrs.loc[estrs['period']==1]['locus']))]
table_sum=pd.DataFrame.from_dict(table)
table_sum.index=specs
table_sum


Total STR loci in introns region: 73637
Total eSTRs found in introns region: 12224

Total STR loci in coding region: 207
Total eSTRs found in coding region: 75

Total STR loci in 5utr region: 641
Total eSTRs found in 5utr region: 212

Total STR loci in 3utr region: 1743
Total eSTRs found in 3utr region: 520

Total STR loci in exon region: 0
Total eSTRs found in exon region: 0


,3utr,5utr,All.loci,coding,exon,introns
Num.str,1743,641,125758,207,0,73637
Num.estr,520,212,20192,75,0,12224
Num.hom,741,162,48240,10,0,28878
Num.e.hom,195,40,8092,2,0,5086


In [26]:
# score distribution of eSTRs at locartion
# examples estrs in location

for i in [1,2,3,4,5,6]:
    print(feature_count[3].loc[feature_count[3]['period']==i].shape )
#A = feature_count[1][['locus','motif','period']].drop_duplicates()    
A = feature_count[1]
A.loc[(A['period']==6)&(A['E.tissues']>=1)]

(6350, 78)
(5074, 78)
(866, 78)
(1684, 78)
(544, 78)
(346, 78)


,chrom,str.start,str.end,gene,str.id,E.tissues,ll.E.tissues,eSTR.Adipose.S,qval.Adipose.S,llqval.Adipose.S,...,qval.Thyroid,llqval.Thyroid,beta.Thyroid,eSTR.Blood,qval.Blood,llqval.Blood,beta.Blood,period,motif,locus
337,chr11,6411931,6411971,ENSG00000166311.5,STR_6411931,4,0,0.0,0.000000,0.233721,...,0.001132,0.000415,0.273466,0.0,0.633439,0.250214,0.123338,6,AGCGCC,chr11:6411931
338,chr11,6411931,6411971,ENSG00000166311.5,STR_6411931,4,0,0.0,0.000000,0.233721,...,0.001132,0.000415,0.273466,0.0,0.633439,0.250214,0.123338,6,AGCGCC,chr11:6411931
339,chr11,6411931,6411971,ENSG00000166311.5,STR_6411931,4,0,0.0,0.000000,0.233721,...,0.001132,0.000415,0.273466,0.0,0.633439,0.250214,0.123338,6,AGCGCC,chr11:6411931
365,chr11,124750440,124750478,ENSG00000154134.10,STR_124750440,6,0,0.0,0.000000,0.508024,...,0.000848,0.000253,0.330100,0.0,NaN,0.265047,0.121464,6,AGCCGG,chr11:124750440
366,chr11,124750440,124750478,ENSG00000154134.10,STR_124750440,6,0,0.0,0.000000,0.508024,...,0.000848,0.000253,0.330100,0.0,NaN,0.265047,0.121464,6,AGCCGG,chr11:124750440
367,chr11,124750440,124750478,ENSG00000154134.10,STR_124750440,6,0,0.0,0.000000,0.508024,...,0.000848,0.000253,0.330100,0.0,NaN,0.265047,0.121464,6,AGCCGG,chr11:124750440
368,chr11,124750440,124750478,ENSG00000154134.10,STR_124750440,6,0,0.0,0.000000,0.508024,...,0.000848,0.000253,0.330100,0.0,NaN,0.265047,0.121464,6,AGCCGG,chr11:124750440
369,chr11,124750440,124750478,ENSG00000154134.10,STR_124750440,6,0,0.0,0.000000,0.508024,...,0.000848,0.000253,0.330100,0.0,NaN,0.265047,0.121464,6,AGCCGG,chr11:124750440
667,chr14,21560752,21560791,ENSG00000165804.11,STR_21560752,1,0,1.0,0.037143,0.007404,...,0.000000,0.275435,0.123603,0.0,NaN,0.407935,-0.100978,6,AGCCTC,chr14:21560752
668,chr14,21560752,21560791,ENSG00000165804.11,STR_21560752,1,0,1.0,0.037143,0.007404,...,0.000000,0.275435,0.123603,0.0,NaN,0.407935,-0.100978,6,AGCCTC,chr14:21560752


In [ ]:
{1, 2, 3, 4, 5, 6}  step
 4, 2, 161, 11, 41   # of str at coding exons
69, 117,195,69,53,54
651---